
Let's remind ourselves of the multiple linear regression setup.

We presume a data generating process of the form

$f(\vec{x}) = \vec{\beta} \cdot \vec{x} + \epsilon$

where $\vec{x}$ has been augmented with an initial $1$ and $\epsilon \sim \mathcal{N}(0, \sigma^2)$.

We have data $(\vec{x}_i, y_i)$ for $i = 1, 2, 3, ..., n$ which we package into an $n \times (p+1)$ design matrix $X$ and an outcome vector $\vec{y}$.

We give a very condensed derivation of the confidence intervals for the parameters here.  A fuller derivation is given in the math hour notes.

Our parameter point estimates are given by the normal equations:

$$
\begin{align*}
\hat{\beta} 
&= (X^\top X)^{-1}X^\top \vec{y}\\
&= (X^\top X)^{-1}X^\top (X \beta + \vec{\epsilon})\\
&= \beta + (X^\top X)^{-1}X^\top \vec{\epsilon} 
\end{align*}
$$

Since $\vec{\epsilon} \sim \mathcal{N}(0, \sigma^2 I)$ then $\hat{\beta}$ is also a multivariate normal distribution.  We can compute the covariance matrix as follows:


$$
\begin{align*}
\operatorname{Cov}(\hat{\beta}) 
&= \operatorname{Cov}((X^\top X)^{-1}X^\top \vec{\epsilon})\\
&= (X^\top X)^{-1}X^\top\operatorname{Cov}(\vec{\epsilon})((X^\top X)^{-1}X^\top)^\top\\
&= (X^\top X)^{-1}X^\top \sigma^2 I X (X^\top X)^{-1}\\
&= \sigma^2 (X^\top X)^{-1}
\end{align*}
$$

Thus we have $\hat{\beta} \sim \mathcal{N}(\beta, \sigma^2 (X^\top X)^{-1})$.  For each individual parameter $\beta_i$ we then have

$$
\frac{\hat{\beta}_{i}-\beta_{i}}{\sigma\sqrt{(X^{T}X)^{-1}_{ii}}} \sim N(0,1)
$$

Leaving the full details to math hour, we then replace the unknown $\sigma^2$ with an unbiased estimate $s^2 = \frac{1}{n-(p+1)} |\vec{y} - \hat{y}|^2$.  This leads to

$$
\frac{\hat{\beta}_{i}-\beta_{i}}{s\sqrt{(X^{T}X)^{-1}_{ii}}} \sim t_{n-(p+1)}
$$

so we obtain $1-\alpha$ confidence intervals of

$$
\hat{\beta}_i \pm t_{1-\alpha/2, n- (p+1)} s \sqrt{(X^{T}X)^{-1}_{ii}}
$$

In [1]:
import numpy as np
from scipy import stats
from statsmodels.regression import linear_model

# generating fake data with just 10 observations
# statsmodels linear_model requires X to have an initial column of ones.
nobs = 10
X = np.random.randn(nobs,3)
X[:,0] = 1
y = np.dot(X, [3,5,-1]) + np.random.randn(nobs)

# fitting an OLS linear model.
# Notice that statsmodels has a different API than scikit-learn
# When we instantiate the model we give it the data in the order (y,X)
# We then fit the model.  Both of these conventions are "backwards" compared to scikit-learn.
model = linear_model.OLS(y, X)
results = model.fit()

In [2]:
#These are the confidence intervals for the model parameters.  
results.conf_int()

array([[ 2.63743129,  3.74731654],
       [ 4.77021259,  5.6820084 ],
       [-1.32407739, -0.3552143 ]])

In [3]:
import numpy as np
import scipy.stats as stats

class LinearRegressionWithCI:
    def __init__(self):
        self.coefficients = None
        self.cov_matrix = None
        self.y_pred = None
        self.residuals = None
        self.mse = None
        self.se = None

    def fit(self, X, y):
        # Add intercept term
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        
        # Implementing the normal equations
        XtX_inv = np.linalg.inv(X.T @ X)
        self.coefficients = XtX_inv @ X.T @ y
        
        self.y_pred = X @ self.coefficients
        self.residuals = y - self.y_pred

        # Degrees of freedom
        df = self.residuals.shape[0] - self.coefficients.shape[0]

        self.mse = np.sum(self.residuals**2) / df
        self.cov_matrix = self.mse * XtX_inv
        self.se = np.sqrt(np.diag(self.cov_matrix))

    def confidence_intervals(self, alpha=0.05):
        # Degrees of freedom
        df = self.residuals.shape[0] - self.coefficients.shape[0]
        
        # t-value for the given confidence level
        t_value = stats.t.ppf(1 - alpha/2, df)
        
        # Compute confidence intervals
        intervals = []
        for i in range(len(self.coefficients)):
            lower_bound = self.coefficients[i] - t_value * self.se[i]
            upper_bound = self.coefficients[i] + t_value * self.se[i]
            intervals.append((lower_bound, upper_bound))
        
        return intervals

In [4]:
reg = LinearRegressionWithCI()
reg.fit(X[:,1:], y)
print(reg.confidence_intervals())
print("\n")
print(results.conf_int())

[(2.637431290479213, 3.747316535099524), (4.770212591057406, 5.682008399047496), (-1.3240773928350236, -0.35521429630848705)]


[[ 2.63743129  3.74731654]
 [ 4.77021259  5.6820084 ]
 [-1.32407739 -0.3552143 ]]


In [5]:
import numpy as np
import statsmodels.api as sm

# Parameters for the simulation
n_simulations = 1000
n_samples = 100
true_intercept = 2
true_slope = 3

# Storage for results
intercept_within_ci = 0
slope_within_ci = 0

# Run the simulations
for _ in range(n_simulations):
    # Simulate data
    x = np.random.normal(size=n_samples)
    eps = np.random.normal(size=n_samples)
    y = true_intercept + true_slope * x + eps
    
    # Fit the model
    x_with_const = sm.add_constant(x)  # Adds intercept term to the model
    model = sm.OLS(y, x_with_const)
    results = model.fit()
    
    # Get the confidence intervals
    ci = results.conf_int(alpha=0.05)
    
    # Check if true parameters are within the CIs
    if ci[0, 0] <= true_intercept <= ci[0, 1]:
        intercept_within_ci += 1
    if ci[1, 0] <= true_slope <= ci[1, 1]:
        slope_within_ci += 1

# Calculate the proportion of times the true parameters were within the confidence intervals
intercept_coverage = intercept_within_ci / n_simulations
slope_coverage = slope_within_ci / n_simulations

intercept_coverage, slope_coverage

(0.957, 0.963)